In [ ]:
//////////////view 接口函数//////////////
///@notice get tokenA reserves
    function tokenAReserves() public view override returns (uint256) {
        return reserveMap[tokenA];
    }

///@notice get tokenB reserves
    function tokenBReserves() public view override returns (uint256) {
        return reserveMap[tokenB];
    }


///@notice get user order details
    function getOrderDetails(uint256 orderId)
        external
        view
        returns (LongTermOrdersLib.Order memory)
    {
        return longTermOrders.orderMap[orderId];
    }

///@notice returns the user order reward factor at submission
    function getOrderRewardFactorAtSubmission(uint256 orderId)
        external
        view
        override
        returns (uint256 orderRewardFactorAtSubmission)
    {
        address orderSellToken = longTermOrders.orderMap[orderId].sellTokenId;

        orderRewardFactorAtSubmission = longTermOrders
            .OrderPoolMap[orderSellToken]
            .rewardFactorAtSubmission[orderId];
    }

///@notice returns the current state of the twamm
    function getTWAMMState()
        external
        view
        override
        returns (
            uint256 lastVirtualOrderBlock,
            uint256 tokenASalesRate,
            uint256 tokenBSalesRate,
            uint256 orderPoolARewardFactor,
            uint256 orderPoolBRewardFactor
        )
    {
        lastVirtualOrderBlock = longTermOrders.lastVirtualOrderBlock;
        tokenASalesRate = longTermOrders.OrderPoolMap[tokenA].currentSalesRate;
        tokenBSalesRate = longTermOrders.OrderPoolMap[tokenB].currentSalesRate;
        orderPoolARewardFactor = longTermOrders
            .OrderPoolMap[tokenA]
            .rewardFactor;
        orderPoolBRewardFactor = longTermOrders
            .OrderPoolMap[tokenB]
            .rewardFactor;
    }

///@notice returns cumulative sales rate of orders ending on this block number
    function getTWAMMSalesRateEnding(uint256 blockNumber)
        external
        view
        override
        returns (
            uint256 orderPoolASalesRateEnding,
            uint256 orderPoolBSalesRateEnding
        )
    {
        orderPoolASalesRateEnding = longTermOrders
            .OrderPoolMap[tokenA]
            .salesRateEndingPerBlock[blockNumber];
        orderPoolBSalesRateEnding = longTermOrders
            .OrderPoolMap[tokenB]
            .salesRateEndingPerBlock[blockNumber];
    }

In [ ]:
//////////////虚拟订单结算公式//////////////
def computeVirtualBalances(
        tokenAStart,
        tokenBStart,
        tokenAIn,
        tokenBIn
    ) returns (
            tokenAOut,
            tokenBOut,
            ammEndTokenA,
            ammEndTokenB
        )
    {
        //if no tokens are sold to the pool, we don't need to execute any orders
        if (tokenAIn == 0 && tokenBIn == 0) {
            tokenAOut = 0;
            tokenBOut = 0;
            ammEndTokenA = tokenAStart;
            ammEndTokenB = tokenBStart;
        }
        //in the case where only one pool is selling, we just perform a normal swap
        else if (tokenAIn == 0) {
            //constant product formula
            tokenAOut = (tokenAStart * tokenBIn) / (tokenBStart + tokenBIn);
            tokenBOut = 0;
            ammEndTokenA = tokenAStart - tokenAOut;
            ammEndTokenB = tokenBStart + tokenBIn;
        } else if (tokenBIn == 0) {
            tokenAOut = 0;
            //constant product formula
            tokenBOut = (tokenBStart * tokenAIn) / (tokenAStart + tokenAIn);
            ammEndTokenA = tokenAStart + tokenAIn;
            ammEndTokenB = tokenBStart - tokenBOut;
        }
        //when both pools sell, we use the TWAMM formula
        else {
            k = tokenAStart * tokenBStart;
            c = computeC(tokenAStart, tokenBStart, tokenAIn, tokenBIn);
            endA = computeAmmEndTokenA(tokenAIn, tokenBIn, c, k, tokenAStart, tokenBStart);
            endB = tokenAStart * tokenBStart / endA;
            
            outA = tokenAStart + tokenAIn - endA;
            outB = tokenBStart + tokenBIn - endB;
            require(outA >= 0 && outB >= 0)
            
            return (
                outA,
                outB,
                endA,
                endB
            );
        }
    }

def computeC(
        tokenAStart,
        tokenBStart,
        tokenAIn,
        tokenBIn
    ) returns(c) {
        c1 = tokenAStart.sqrt() * (tokenBIn.sqrt());
        c2 = tokenBStart.sqrt() * (tokenAIn.sqrt());
        cNumerator = c1 - c2;
        cDenominator = c1 + c2;
        c = cNumerator / cDenominator;
    }

def computeAmmEndTokenA(
        tokenAIn,
        tokenBIn,
        c,
        k,
        tokenAStart,
        tokenBStart
    ) returns (ammEndTokenA) {
        eNumerator = (4 * tokenAIn).sqrt() * (tokenBIn.sqrt());
        eDenominator = tokenAStart.sqrt() * (tokenBStart.sqrt());
        exponent = (eNumerator / eDenominator).exp();
        require(exponent > abs(c));
        fraction = (exponent + c) / (exponent - c);
        scaling = (k / tokenBIn).sqrt() * (tokenAIn.sqrt());
        ammEndTokenA = fraction * scaling;
    }

In [ ]:
//////////////执行虚拟订单结算到指定 blockNumber//////////////
def executeVirtualOrders(blockNumber) {
//变量 
orderBlockInterval = 5;
blockNumber;

tokenA;
tokenB;

reserveA;
reserveB;

lastVirtualOrderBlock;
currentSalesRateA;
currentSalesRateB;
rewardFactorA;
rewardFactorB;

//通过 view 函数获取变量值
reserveA = tokenAReserves();
reserveB = tokenBReserves();
[lastVirtualOrderBlock, currentSalesRateA, currentSalesRateB, rewardFactorA, rewardFactorB] = getTWAMMState();      

//执行虚拟订单结算到 blockNumber
require(blockNumber <= block.number);
lastExpiryBlock = lastVirtualOrderBlock - (lastVirtualOrderBlock % orderBlockInterval) 
n = (blockNumber - lastExpiryBlock) % orderBlockInterval;

if n>=1
for i=1, i++1, i<=n;
    iExpiryBlock = lastExpiryBlock + i * orderBlockInterval;
    [iOrderPoolASalesRateEnding, iOrderPoolBSalesRateEnding] = getTWAMMSalesRateEnding(iExpiryBlock);

    if iOrderPoolASalesRateEnding >0 || iOrderPoolASalesRateEnding >0;
    //amount sold from virtual trades
    blockNumberIncrement = iExpiryBlock - lastVirtualOrderBlock;
    tokenASellAmount = currentSalesRateA * blockNumberIncrement;
    tokenBSellAmount = currentSalesRateB * blockNumberIncrement;

    //updated balances from sales
    (
        tokenAOut,
        tokenBOut,
        ammEndTokenA,
        ammEndTokenB
    ) = computeVirtualBalances(
        reserveA,
        reserveB,
        tokenASellAmount,
        tokenBSellAmount
    );

    //update state
    reserveA = ammEndTokenA;
    reserveB = ammEndTokenB;
    
    lastVirtualOrderBlock = iExpiryBlock;
    currentSalesRateA = currentSalesRateA - iOrderPoolASalesRateEnding;
    currentSalesRateB = currentSalesRateB - iOrderPoolBSalesRateEnding;
    
    if (currentSalesRateA != 0) {rewardFactorA = rewardFactorA + tokenBOut / currentSalesRateA;}    
    if (currentSalesRateB != 0) {rewardFactorB = rewardFactorB + tokenAOut / currentSalesRateB;}
    
    //finally, move state to blockNumber if necessary
    [endOrderPoolASalesRateEnding, endOrderPoolBSalesRateEnding] = getTWAMMSalesRateEnding(blockNumber);
    blockNumberIncrement = blockNumber - lastVirtualOrderBlock;
    tokenASellAmount = currentSalesRateA * blockNumberIncrement;
    tokenBSellAmount = currentSalesRateB * blockNumberIncrement;    
    
    //updated balances from sales
    (
        tokenAOut,
        tokenBOut,
        ammEndTokenA,
        ammEndTokenB
    ) = computeVirtualBalances(
        reserveA,
        reserveB,
        tokenASellAmount,
        tokenBSellAmount
    );

    //update state
    reserveA = ammEndTokenA;
    reserveB = ammEndTokenB;
    
    lastVirtualOrderBlock = blockNumber;
    currentSalesRateA = currentSalesRateA - endOrderPoolASalesRateEnding;
    currentSalesRateB = currentSalesRateB - endOrderPoolBSalesRateEnding;
    
    if (currentSalesRateA != 0) {rewardFactorA = rewardFactorA + tokenBOut / currentSalesRateA;}   
    if (currentSalesRateB != 0) {rewardFactorB = rewardFactorB + tokenAOut / currentSalesRateB;}
else
    [endOrderPoolASalesRateEnding, endOrderPoolBSalesRateEnding] = getTWAMMSalesRateEnding(blockNumber);
    blockNumberIncrement = blockNumber - lastVirtualOrderBlock;
    tokenASellAmount = currentSalesRateA * blockNumberIncrement;
    tokenBSellAmount = currentSalesRateB * blockNumberIncrement;    
    
    //updated balances from sales
    (   
        tokenAOut,
        tokenBOut,
        ammEndTokenA,
        ammEndTokenB
    ) = computeVirtualBalances(
        reserveA,
        reserveB,
        tokenASellAmount,
        tokenBSellAmount
    );

    //update state
    reserveA = ammEndTokenA;
    reserveB = ammEndTokenB;
    
    lastVirtualOrderBlock = blockNumber;
    currentSalesRateA = currentSalesRateA - endOrderPoolASalesRateEnding;
    currentSalesRateB = currentSalesRateB - endOrderPoolBSalesRateEnding;
    
    if (currentSalesRateA != 0) {rewardFactorA = rewardFactorA + tokenBOut / currentSalesRateA;}   
    if (currentSalesRateB != 0) {rewardFactorB = rewardFactorB + tokenAOut / currentSalesRateB;}
}

In [ ]:
//////////////执行虚拟订单结算到 current block, 获取真实 reserve 和 sales rate//////////////
executeVirtualOrders(block.number);
  return 
    reserveA;
    reserveB;
    currentSalesRateA;
    currentSalesRateB;

In [ ]:
//////////////执行虚拟订单结算, 获取订单 withdrawableProceeds//////////////
//通过 view 函数获取 order 数据
struct Order {
  uint256 id;
  uint256 expirationBlock;
  uint256 saleRate;
  address owner;
  address sellTokenId;
  address buyTokenId;
} = getOrderDetails(uint256 orderId);

//变量
withdrawableProceeds;
orderRewardFactorAtSubmission = getOrderRewardFactorAtSubmission(uint256 orderId);

if block.number >= expirationBlock;
//执行虚拟订单结算到 expirationBlock
executeVirtualOrders(expirationBlock);
  return 
    rewardFactorA;
    rewardFactorB;    
  if sellTokenId == tokenA;
  withdrawableProceeds = (rewardFactorA - orderRewardFactorAtSubmission) * saleRate * 97 / 100;
  if sellTokenId == tokenB;
  withdrawableProceeds = (rewardFactorB - orderRewardFactorAtSubmission) * saleRate * 97 / 100;

if block.number < expirationBlock;
//执行虚拟订单结算到 block.number
executeVirtualOrders(block.number);
  return 
    rewardFactorA;
    rewardFactorB;
  if sellTokenId == tokenA;
  withdrawableProceeds = (rewardFactorA - orderRewardFactorAtSubmission) * saleRate * 97 / 100;
  if sellTokenId == tokenB;
  withdrawableProceeds = (rewardFactorB - orderRewardFactorAtSubmission) * saleRate * 97 / 100;